<a href="https://colab.research.google.com/github/jooalee64/mastertip/blob/main/masttip_experiments_tuned_paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MastTip — Tuned Colab (Path-aware for labels-storage/stage-b-labels)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

import os, math, glob, random
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import numpy as np
from PIL import Image, ImageDraw
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.ops import roi_align
from torchvision.transforms import functional as TF
from torchvision.models import resnet50, ResNet50_Weights

ROOT_DIR = "/gdrive/MyDrive/boats"
IMAGES_DIR = os.path.join(ROOT_DIR, "yolo", "images")
TRAIN_IMAGES_DIR = os.path.join(IMAGES_DIR, "train")
VAL_IMAGES_DIR   = os.path.join(IMAGES_DIR, "val")
TEST_IMAGES_DIR  = os.path.join(IMAGES_DIR, "test")

LABELS_STORAGE = os.path.join(ROOT_DIR, "yolo", "labels-storage")
STAGE_B_DIR    = os.path.join(LABELS_STORAGE, "stage-b-labels")
HULL_LABELS_DIR = os.path.join(STAGE_B_DIR, "hull-labels")
TIP_LABELS_DIR  = os.path.join(STAGE_B_DIR, "tip-label")
SAILBOAT_LABELS_DIR = os.path.join(STAGE_B_DIR, "sailboat-labels")  # optional

LABELS_SPLIT_BY_SET = False # Changed from True to False

MAX_SIDE    = 1024
BATCH_SIZE  = 4
EPOCHS      = 8
LR          = 2e-4
WEIGHT_DECAY = 5e-4
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

ROI_OUT   = 14
UP_OUT    = 56
ROI_EXPAND = dict(alpha_up=1.6, beta_down=0.10, gamma_side=0.15)
CONE_CFG = dict(base_y_ratio=0.80, apex_y_ratio=0.02, base_width_ratio=0.40, lateral_bias=0.0)

USE_CONE_MASK = True
USE_GAUSS_NLL = True
USE_REFINEMENT = True

def set_seed(seed=1337):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(1337)

print("Using labels from:", HULL_LABELS_DIR, "and", TIP_LABELS_DIR)
print("DEVICE:", DEVICE)

Mounted at /gdrive
Using labels from: /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels and /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/tip-label
DEVICE: cpu


In [ ]:
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def list_images(folder: str) -> List[str]:
    out = []
    for ext in IMG_EXTS:
        out += glob.glob(os.path.join(folder, f"*{ext}"))
    return sorted(out)

@dataclass
class LetterboxInfo:
    scale: float
    pad: Tuple[int, int]
    new_size: Tuple[int, int]
    orig_size: Tuple[int, int]

def letterbox(img, max_side: int = 1024):
    w, h = img.size
    s = min(max_side / max(h, w), 1.0)
    nw, nh = int(round(w * s)), int(round(h * s))
    img_r = img.resize((nw, nh), Image.BILINEAR)
    dw, dh = max_side - nw, max_side - nh
    left = dw // 2; top = dh // 2
    canvas = Image.new("RGB", (max_side, max_side), (114,114,114))
    canvas.paste(img_r, (left, top))
    info = LetterboxInfo(scale=s, pad=(left, top), new_size=(nw, nh), orig_size=(w, h))
    return canvas, info

def yolo_box_to_xyxy(yolo, img_w, img_h):
    _, cx, cy, bw, bh = yolo
    cx *= img_w; cy *= img_h; bw *= img_w; bh *= img_h
    x1 = cx - bw/2; y1 = cy - bh/2; x2 = cx + bw/2; y2 = cy + bh/2
    return [x1, y1, x2, y2]

def yolo_center(yolo, img_w, img_h):
    _, cx, cy, _, _ = yolo
    return [cx * img_w, cy * img_h]

def find_label_for_image(img_path: str, label_root: str, split_by_set=True) -> Optional[str]:
    base = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
    p = img_path.replace("\\","/")
    if split_by_set:
        for split in ["train","val","test"]:
            if f"/{split}/" in p:
                cand = os.path.join(label_root, split, base)
                if os.path.exists(cand): return cand
        for split in ["train","val","test"]:
            cand = os.path.join(label_root, split, base)
            if os.path.exists(cand): return cand
    cand = os.path.join(label_root, base)
    return cand if os.path.exists(cand) else None

In [ ]:
class MastTipDatasetSeparate(Dataset):
    def __init__(self, img_dir: str, hull_root: str, tip_root: str, max_side: int = 1024, split_by_set=True):
        self.img_paths = list_images(img_dir)
        self.hull_root = hull_root
        self.tip_root = tip_root
        self.max_side = max_side
        self.split_by_set = split_by_set # This will be effectively False now based on the global variable
        self.split_name = os.path.basename(img_dir) # Infer split name from image directory

    def __len__(self): return len(self.img_paths)

    def _load_one_file(self, path: str, img_w: int, img_h: int, as_points=False):
        items = []
        if (path is None) or (not os.path.exists(path)): return items
        # Assuming CSV format with columns: class_id, center_x, center_y, width, height
        import pandas as pd
        try:
            # Read only the first 5 columns to avoid "too many values to unpack" error
            df = pd.read_csv(path, header=None).iloc[:, :5]
            for index, row in df.iterrows():
                # Assuming the format is class_id, center_x, center_y, width, height
                # We only care about the bounding box/point coordinates
                vals = row.tolist()
                if len(vals) < 5: continue # Skip if not enough columns
                if as_points: items.append(yolo_center(vals, img_w, img_h))
                else:         items.append(yolo_box_to_xyxy(vals, img_w, img_h))
        except Exception as e:
            print(f"Error loading {path}: {e}")
            return [] # Return empty list if there's an error reading the CSV
        return items


    def find_label_for_image_csv(self, img_path: str, label_root: str, label_type: str) -> Optional[str]:
        # Construct the expected CSV filename based on split and label type
        split_name = self.split_name # e.g., "train", "val", "test"
        # Adjust label_type to match CSV file naming convention
        if label_type == "hull":
            csv_filename = f"hull-{split_name}.csv"
        elif label_type == "tip":
            # The user's example shows "test-tips-csv.csv", so handle this
            if split_name == "test":
                csv_filename = "test-tips-csv.csv"
            elif split_name == "train":
                csv_filename = "train-tips-csv.csv"
            elif split_name == "val":
                 # Assuming a similar pattern for val, adjust if necessary
                csv_filename = "val-tips-csv.csv"
            else:
                 # Fallback or error if split name is unexpected
                return None
        elif label_type == "sailboat":
             # Assuming a similar pattern for sailboat, adjust if necessary
            csv_filename = f"sailboat-{split_name}.csv"
        else:
            return None # Unknown label type

        # Construct the full path to the CSV file
        # Assuming the CSVs are directly under the label_root for hull and tip,
        # and under sailboat-labels for sailboat
        if label_type in ["hull", "tip"]:
             cand = os.path.join(label_root, csv_filename)
        elif label_type == "sailboat":
             # Assuming sailboat labels are in a subdirectory like sailboat-labels
             cand = os.path.join(label_root, "sailboat-labels", csv_filename)
        else:
            return None


        return cand if os.path.exists(cand) else None


    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        # Use the new function to find CSV labels
        hull_lab = self.find_label_for_image_csv(img_path, self.hull_root, "hull")
        tip_lab  = self.find_label_for_image_csv(img_path, self.tip_root,  "tip")

        hulls = self._load_one_file(hull_lab, w, h, as_points=False)
        tips  = self._load_one_file(tip_lab,  w, h, as_points=True)

        img_lb, info = letterbox(img, MAX_SIDE)

        def map_xyxy(b):
            x1,y1,x2,y2 = b
            return [x1*info.scale+info.pad[0], y1*info.scale+info.pad[1],
                    x2*info.scale+info.pad[0], y2*info.scale+info.pad[1]]
        def map_point(p):
            x,y=p; return [x*info.scale+info.pad[0], y*info.scale+info.pad[1]]

        hulls_lb = [map_xyxy(b) for b in hulls]
        tips_lb  = [map_point(p) for p in tips]

        img_t = TF.to_tensor(img_lb)

        return dict(
            image=img_t,
            img_path=img_path,
            hulls=torch.tensor(hulls_lb, dtype=torch.float32),
            tips=torch.tensor(tips_lb, dtype=torch.float32),
            lbinfo=None
        )

def collate(batch):
    return dict(
        images=torch.stack([b["image"] for b in batch], 0),
        img_paths=[b["img_path"] for b in batch],
        lbinfo=[b["lbinfo"] for b in batch],
        hulls=[b["hulls"] for b in batch],
        tips=[b["tips"] for b in batch],
    )

In [ ]:

class Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        m = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.stem = nn.Sequential(m.conv1, m.bn1, m.relu, m.maxpool)
        self.layer1, self.layer2, self.layer3, self.layer4 = m.layer1, m.layer2, m.layer3, m.layer4
        self.out_channels = 2048
    def forward(self, x):
        x = self.stem(x); x = self.layer1(x); x = self.layer2(x); x = self.layer3(x); x = self.layer4(x)
        return x

def gaussian_nll_2d(mu, log_sigma, rho_raw, y, reduction="mean", eps=1e-6):
    sigma = torch.exp(log_sigma) + eps
    rho = torch.tanh(rho_raw).clamp(-0.999, 0.999)
    dx = (y[:,0]-mu[:,0]) / sigma[:,0]; dy = (y[:,1]-mu[:,1]) / sigma[:,1]
    rho_sq = rho[:,0]**2; one_m_rho_sq = 1.0 - rho_sq + eps
    q = (dx*dx - 2*rho[:,0]*dx*dy + dy*dy) / one_m_rho_sq
    log_det = 2*log_sigma[:,0] + 2*log_sigma[:,1] + torch.log(one_m_rho_sq)
    nll = 0.5 * (log_det + q + math.log(2.0*math.pi))
    return nll.mean() if reduction=="mean" else nll

class TipHeadGaussian(nn.Module):
    def __init__(self, in_channels=2048, hidden=384):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, hidden, 3, padding=1), nn.ReLU(True),
                                  nn.Conv2d(hidden, hidden, 3, padding=1), nn.ReLU(True),
                                  nn.AdaptiveAvgPool2d(1))
        self.fc = nn.Linear(hidden, 5)
    def forward(self, x):
        h = self.conv(x).flatten(1); out = self.fc(h)
        mu = torch.sigmoid(out[:, :2]); return mu, out[:,2:4], out[:,4:5]

class ShaftRefineHead(nn.Module):
    def __init__(self, in_channels=2048, mid=128):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, mid, 3, padding=1), nn.ReLU(True),
                                  nn.Conv2d(mid, 1, 1))
    def forward(self, x): return torch.sigmoid(self.conv(x))

def make_cone_mask(h, w, base_y_ratio=0.80, apex_y_ratio=0.02, base_width_ratio=0.40, apex_x=0.5, device="cpu"):
    ys, xs = torch.meshgrid(torch.linspace(0,1,h,device=device), torch.linspace(0,1,w,device=device), indexing="ij")
    apex_y = apex_y_ratio; base_y = base_y_ratio; hw_base = base_width_ratio * 0.5
    t = ((ys - apex_y) / (base_y - apex_y)).clamp(0,1); half_w = hw_base * t
    return ((ys >= apex_y) & (ys <= base_y) & (torch.abs(xs - apex_x) <= half_w + 1e-6)).float()

class StageB(nn.Module):
    def __init__(self, roi_out=14, up_out=56, cone_cfg: Dict=None, roi_expand: Dict=None,
                 use_cone=True, use_refine=True):
        super().__init__()
        self.backbone = Backbone(); self.roi_out=roi_out; self.up_out=up_out
        self.head = TipHeadGaussian(self.backbone.out_channels, hidden=384)
        self.refine_head = ShaftRefineHead(self.backbone.out_channels, mid=128) if use_refine else None
        self.stride = 32; self.use_cone = use_cone
        self.cone_cfg = cone_cfg or {}; self.roi_expand = roi_expand or dict(alpha_up=1.6, beta_down=0.10, gamma_side=0.15)
    def _expand(self, x1,y1,x2,y2,W,H):
        w=(x2-x1); h=(y2-y1); up=self.roi_expand["alpha_up"]*h; down=self.roi_expand["beta_down"]*h; side=self.roi_expand["gamma_side"]*w
        xe1=(x1-side).clamp(min=0); xe2=(x2+side).clamp(max=W); ye1=(y1-up).clamp(min=0); ye2=(y2+down).clamp(max=H); return xe1,ye1,xe2,ye2
    def _build_rois(self, batch):
        rois=[]; W=batch["images"].shape[-1]; H=batch["images"].shape[-2]
        for b_idx,hulls in enumerate(batch["hulls"]):
            if len(hulls)==0: continue
            x1=torch.min(hulls[:,0],hulls[:,2]); y1=torch.min(hulls[:,1],hulls[:,3])
            x2=torch.max(hulls[:,0],hulls[:,2]); y2=torch.max(hulls[:,1],hulls[:,3])
            xe1,ye1,xe2,ye2=self._expand(x1,y1,x2,y2,W,H); he=torch.stack([xe1,ye1,xe2,ye2],1)
            idx=torch.full((he.size(0),1), float(b_idx), dtype=he.dtype, device=he.device); rois.append(torch.cat([idx,he],1))
        return torch.cat(rois,0).to(batch["images"].device) if len(rois)>0 else torch.zeros((0,5), dtype=torch.float32, device=batch["images"].device)
    def _apex_x_from_roi(self, rois):
        cx=(rois[:,1]+rois[:,3])/2.0; W=(rois[:,3]-rois[:,1]).clamp(min=1.0)
        apex_x01=(cx-rois[:,1])/W; bias=self.cone_cfg.get("lateral_bias",0.0); return (apex_x01+bias).clamp(0.0,1.0)
    def forward_once(self, images, rois_xyxy):
        feat=self.backbone(images)
        pooled=roi_align(feat, rois_xyxy, output_size=(self.roi_out,self.roi_out), spatial_scale=1.0/self.stride, sampling_ratio=-1, aligned=True)
        up=F.interpolate(pooled, size=(self.up_out,self.up_out), mode="bilinear", align_corners=False)
        if self.use_cone:
            cms=[]
            for ax in self._apex_x_from_roi(rois_xyxy).tolist():
                cms.append(make_cone_mask(self.up_out,self.up_out, apex_x=ax, device=up.device, **{k:v for k,v in self.cone_cfg.items() if k!='lateral_bias'})[None,None])
            cm=torch.cat(cms,0); up=up*cm
        mu01, log_sigma, rho_raw = self.head(up)
        heatmap = self.refine_head(up) if self.refine_head is not None else None
        return mu01, log_sigma, rho_raw, rois_xyxy, heatmap

    def forward(self, batch, targets_roi01):
        images=batch["images"].to(next(self.parameters()).device)
        rois=self._build_rois(batch)
        mu01, log_sigma, rho_raw, rois_xyxy, heatmap = self.forward_once(images, rois)
        loss = gaussian_nll_2d(mu01, log_sigma, rho_raw, targets_roi01.to(mu01.device))
        if self.refine_head is not None:
            N,_,H,W = heatmap.shape; gt=torch.zeros_like(heatmap)
            xs=(targets_roi01[:,0]*(W-1)).long().clamp(0,W-1); ys=(targets_roi01[:,1]*(H-1)).long().clamp(0,H-1)
            for i in range(N): gt[i,0,:ys[i]+1, xs[i]] = 1.0
            loss = loss + 0.1*F.binary_cross_entropy(heatmap, gt)
        return loss, (mu01, log_sigma, rho_raw, rois_xyxy, heatmap)

def tips_to_roi_targets(batch, rois_xyxy, eps=1e-6):
    tips=[t for t in batch["tips"] if len(t)>0]
    if len(tips)==0: return torch.zeros((0,2), dtype=torch.float32, device=batch["images"].device)
    tips=torch.cat(tips,0).to(batch["images"].device)
    x1,y1,x2,y2 = rois_xyxy[:,1], rois_xyxy[:,2], rois_xyxy[:,3], rois_xyxy[:,4]
    w=(x2-x1).clamp(min=1.0); h=(y2-y1).clamp(min=1.0)
    rx=((tips[:,0]-x1)/w).clamp(0.0+eps,1.0-eps); ry=((tips[:,1]-y1)/h).clamp(0.0+eps,1.0-eps)
    return torch.stack([rx,ry],1)

def denorm_roi_to_image(mu01, rois_xyxy):
    x1,y1,x2,y2 = rois_xyxy[:,1], rois_xyxy[:,2], rois_xyxy[:,3], rois_xyxy[:,4]
    w=(x2-x1); h=(y2-y1); return torch.stack([x1+mu01[:,0]*w, y1+mu01[:,1]*h],1)


In [ ]:

train_ds = MastTipDatasetSeparate(TRAIN_IMAGES_DIR, HULL_LABELS_DIR, TIP_LABELS_DIR, max_side=MAX_SIDE, split_by_set=LABELS_SPLIT_BY_SET)
val_ds   = MastTipDatasetSeparate(VAL_IMAGES_DIR,   HULL_LABELS_DIR, TIP_LABELS_DIR, max_side=MAX_SIDE, split_by_set=LABELS_SPLIT_BY_SET)
test_ds  = MastTipDatasetSeparate(TEST_IMAGES_DIR,  HULL_LABELS_DIR, TIP_LABELS_DIR, max_side=MAX_SIDE, split_by_set=LABELS_SPLIT_BY_SET)

print("Num train/val/test:", len(train_ds), len(val_ds), len(test_ds))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate, num_workers=2, pin_memory=True)


Num train/val/test: 1050 225 225


In [ ]:

model = StageB(roi_out=ROI_OUT, up_out=UP_OUT, cone_cfg=CONE_CFG, roi_expand=ROI_EXPAND,
               use_cone=USE_CONE_MASK, use_refine=USE_REFINEMENT).to(DEVICE)
opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

def run_epoch(loader, train=True):
    model.train(train); total, n = 0.0, 0
    for batch in loader:
        batch["images"] = batch["images"].to(DEVICE, non_blocking=True)
        rois = model._build_rois(batch)
        if rois.numel()==0: continue
        targets = tips_to_roi_targets(batch, rois)
        loss, _ = model(batch, targets)
        if train:
            opt.zero_grad(set_to_none=True); loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0); opt.step()
        total += float(loss.detach().cpu()); n += 1
    return total / max(n,1)

for e in range(1, EPOCHS+1):
    tr = run_epoch(train_loader, True); va = run_epoch(val_loader, False)
    print(f"[Epoch {e:02d}] train={tr:.4f}  val={va:.4f}")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 139MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)

Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boat

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Streaming output truncated to the last 5000 lines.
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-train.csv: too many values to unp

In [ ]:

def evaluate(loader):
    model.eval(); rmse_accum=nme_accum=count=0.0
    with torch.no_grad():
        for batch in loader:
            batch["images"] = batch["images"].to(DEVICE, non_blocking=True)
            rois = model._build_rois(batch)
            if rois.numel()==0: continue
            targets = tips_to_roi_targets(batch, rois)
            _, (mu01, log_sigma, rho_raw, rois_out, heatmap) = model(batch, targets)
            pred_xy = denorm_roi_to_image(mu01, rois_out).cpu()
            gt_xy = torch.cat([t for t in batch["tips"] if len(t)>0], 0).cpu()
            diff = pred_xy - gt_xy
            rmse = torch.sqrt((diff[:,0]**2 + diff[:,1]**2) + 1e-9).mean().item()
            x1,y1,x2,y2 = rois_out[:,1].cpu(), rois_out[:,2].cpu(), rois_out[:,3].cpu(), rois_out[:,4].cpu()
            diag = torch.sqrt((x2-x1)**2 + (y2-y1)**2) + 1e-9
            nme = (torch.sqrt((diff[:,0]**2 + diff[:,1]**2)) / diag).mean().item()
            rmse_accum += rmse; nme_accum += nme; count += 1
    return rmse_accum/max(count,1), nme_accum/max(count,1)

rmse, nme = evaluate(test_loader)
print(f"Test RMSE(px): {rmse:.2f} | NME: {nme:.4f}")


Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/hull-test.csv: too many values to unpack (expected 5)
Error loading /gdrive/MyDrive/boats/yolo/

### Debug Cells

In [ ]:
print("Number of images in train_ds:", len(train_ds))
print("Number of images in val_ds:", len(val_ds))
print("Number of images in test_ds:", len(test_ds))

Number of images in train_ds: 1050
Number of images in val_ds: 225
Number of images in test_ds: 225


In [ ]:
# --- 라벨 매칭 상태 전수 점검 ---
import os

def scan_split_paths(ds, split_name):
    n_img = len(ds)
    n_with_h = n_with_t = n_both = 0
    miss_h, miss_t = [], []
    for img_path in ds.img_paths:
        hull_lab = find_label_for_image(img_path, HULL_LABELS_DIR, split_by_set=LABELS_SPLIT_BY_SET)
        tip_lab  = find_label_for_image(img_path, TIP_LABELS_DIR,  split_by_set=LABELS_SPLIT_BY_SET)
        has_h = os.path.exists(hull_lab) if hull_lab else False
        has_t = os.path.exists(tip_lab)  if tip_lab  else False
        n_with_h += int(has_h); n_with_t += int(has_t); n_both += int(has_h and has_t)
        if not has_h and len(miss_h) < 5: miss_h.append((img_path, hull_lab))
        if not has_t and len(miss_t) < 5: miss_t.append((img_path, tip_lab))
    print(f"[{split_name}] images={n_img}  with_hull={n_with_h}  with_tip={n_with_t}  with_both={n_both}")
    if miss_h:
        print("\n-- First missing hull (5) --")
        for img, lab in miss_h: print(os.path.basename(img), "→", lab)
    if miss_t:
        print("\n-- First missing tip (5) --")
        for img, lab in miss_t:  print(os.path.basename(img), "→", lab)

scan_split_paths(train_ds, "train")
scan_split_paths(val_ds,   "val")
scan_split_paths(test_ds,  "test")


[train] images=1050  with_hull=0  with_tip=0  with_both=0

-- First missing hull (5) --
1.png → None
10.png → None
1003.png → None
1004.png → None
1007.png → None

-- First missing tip (5) --
1.png → None
10.png → None
1003.png → None
1004.png → None
1007.png → None
[val] images=225  with_hull=0  with_tip=0  with_both=0

-- First missing hull (5) --
1002.png → None
1005.png → None
1006.png → None
101.png → None
1011.png → None

-- First missing tip (5) --
1002.png → None
1005.png → None
1006.png → None
101.png → None
1011.png → None
[test] images=225  with_hull=0  with_tip=0  with_both=0

-- First missing hull (5) --
100.png → None
1000.png → None
1001.png → None
1014.png → None
102.png → None

-- First missing tip (5) --
100.png → None
1000.png → None
1001.png → None
1014.png → None
102.png → None


### Check for missing label files

In [ ]:
import os

missing_hull_files_to_check = [
    os.path.join(HULL_LABELS_DIR, "1.txt"),
    os.path.join(HULL_LABELS_DIR, "10.txt"),
    os.path.join(HULL_LABELS_DIR, "1003.txt"),
]

missing_tip_files_to_check = [
    os.path.join(TIP_LABELS_DIR, "1.txt"),
    os.path.join(TIP_LABELS_DIR, "10.txt"),
    os.path.join(TIP_LABELS_DIR, "1003.txt"),
]

print("Checking for missing hull label files:")
for fpath in missing_hull_files_to_check:
    exists = os.path.exists(fpath)
    print(f"{fpath}: {exists}")

print("\nChecking for missing tip label files:")
for fpath in missing_tip_files_to_check:
    exists = os.path.exists(fpath)
    print(f"{fpath}: {exists}")

Checking for missing hull label files:
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/1.txt: False
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/10.txt: False
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/hull-labels/1003.txt: False

Checking for missing tip label files:
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/tip-label/1.txt: False
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/tip-label/10.txt: False
/gdrive/MyDrive/boats/yolo/labels-storage/stage-b-labels/tip-label/1003.txt: False
